# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [2]:
geo_df_01 = pd.read_csv('datasets/geo_data_0.csv')
geo_df_02 = pd.read_csv('datasets/geo_data_1.csv')
geo_df_03 = pd.read_csv('datasets/geo_data_2.csv')

**Описание таблицы**

* `id` — уникальный идентификатор скважины;
* `f0, f1, f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
* `product` — объём запасов в скважине (тыс. баррелей).


In [3]:
# функция для обработки данных датафрейма
def research(df):
    
    display(df)
    print('Информация по таблице:', '\n')
    df.info()
    print()
    print('Количество пропусков:', '\n')
    print(df.isna().sum(), '\n', '\n')
    print('Процент пропусков:', '\n')
    print(df.isna().mean()*100, '\n')

    display(df.describe())
    print()
    print('Число явных дубликатов:', df.duplicated().sum(), '\n')




### Данные по 1-му месторождению

In [4]:
research(geo_df_01)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


Информация по таблице: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество пропусков: 

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 
 

Процент пропусков: 

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64 



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Число явных дубликатов: 0 



### Данные по 2-му месторождению

In [5]:
research(geo_df_02)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


Информация по таблице: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество пропусков: 

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 
 

Процент пропусков: 

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64 



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



Число явных дубликатов: 0 



### Данные по 3-му месторождению

In [6]:
research(geo_df_03)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


Информация по таблице: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество пропусков: 

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 
 

Процент пропусков: 

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64 



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838



Число явных дубликатов: 0 



**Промежуточный вывод**

Результат проверки: все хорошо
* данные без явных дубликатов
* без пропусков
* нормальное название столбцов
* подходящие типы данных

п.с. можно удалить столбец с id, так как он не влияеет на данные (либо просто не учитывать его при обучении)

In [7]:
geo_df_01 = geo_df_01.drop(['id'], axis=1)
geo_df_02 = geo_df_02.drop(['id'], axis=1)
geo_df_03 = geo_df_03.drop(['id'], axis=1)

## Обучение и проверка модели

**По условиям задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).


* Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
* Обучите модель и сделайте предсказания на валидационной выборке.
* Сохраните предсказания и правильные ответы на валидационной выборке.
* Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
* Проанализируйте результаты.

In [8]:
def region_prediction(df):
   
    features = df.drop(["product"], axis=1)
    target = df["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    scaler = StandardScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    model_lr = LinearRegression(normalize=False)
    model_lr.fit(features_train, target_train)
    predictions = model_lr.predict(features_valid)
    predictions = pd.Series(predictions)
    rmse = np.sqrt(mean_squared_error(target_valid, predictions))  
    stock_mean = df['product'].mean()
    stock_mean_pred = predictions.mean()
    return predictions, rmse, stock_mean, stock_mean_pred, target_valid.reset_index(drop=True)

### Предсказание среднего запаса по 1-му региону

In [9]:
pred_1, rmse_1, stock_mean_1, stock_mean_pred_1, target_valid_1 = region_prediction(geo_df_01)
print('RMSE модели в регионе №1 = {:.3f}'.format(rmse_1))
print('Средний запас предсказанного сырья в регионе №1 = {:.3f} т. баррелей'.format(stock_mean_pred_1))

RMSE модели в регионе №1 = 37.579
Средний запас предсказанного сырья в регионе №1 = 92.593 т. баррелей


C:\Users\Bazunov_Anton\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


### Предсказание среднего запаса по 2-му региону

In [10]:
pred_2, rmse_2, stock_mean_2, stock_mean_pred_2, target_valid_2 = region_prediction(geo_df_02)
print('RMSE модели в регионе №2 = {:.3f}'.format(rmse_2))
print('Средний запас предсказанного сырья в регионе №2 = {:.3f} т. баррелей'.format(stock_mean_pred_2))

RMSE модели в регионе №2 = 0.893
Средний запас предсказанного сырья в регионе №2 = 68.729 т. баррелей


C:\Users\Bazunov_Anton\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


### Предсказание среднего запаса по 3-му региону

In [11]:
pred_3, rmse_3, stock_mean_3, stock_mean_pred_3, target_valid_3 = region_prediction(geo_df_03)
print('RMSE модели в регионе №3 = {:.3f}'.format(rmse_3))
print('Средний запас предсказанного сырья в регионе №3 = {:.3f} т. баррелей'.format(stock_mean_pred_3))

RMSE модели в регионе №3 = 40.030
Средний запас предсказанного сырья в регионе №3 = 94.965 т. баррелей


C:\Users\Bazunov_Anton\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


**Промежуточный вывод:**


Наибольший средний запас 94.965 т. баррелей в 3-ем регионе

## Подготовка к расчёту прибыли

**По условиям задачи:**
* total_points = При разведке региона исследуют 500 точек, из которых с помощью машинного обучения 
* best_points = выбирают 200 лучших для разработки.
* budget = Бюджет на разработку скважин в регионе — 10 млрд рублей.
* income = При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* risk_assessment = После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

* Все ключевые значения для расчётов сохраните в отдельных переменных.
* Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
* Напишите выводы по этапу подготовки расчёта прибыли.

In [12]:
# согласно условию задачи
total_points = 500
best_points = 200
budget = 10000000000
income = 450000
risk_assessment = 0.025

In [13]:
budget_of_one_point = budget/best_points
print('Бюджет бурения одного месторождения, руб:', budget_of_one_point)

Бюджет бурения одного месторождения, руб: 50000000.0


In [14]:
print('Объём сырья для безубыточной разработки новой скважины  = {:.3f} т. баррелей'.format(budget_of_one_point / income))

Объём сырья для безубыточной разработки новой скважины  = 111.111 т. баррелей


**Промежуточный вывод:**
* Бюджет бурения одного месторождения, руб: 50000000.0
* Объём сырья для безубыточной разработки новой скважины  = 111.111 т. баррелей

## Функция расчёта прибыли по выбранным скважинам и предсказание модели

Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
* Выберите скважины с максимальными значениями предсказаний.
* Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
* Рассчитайте прибыль для полученного объёма сырья.

In [15]:
def profit(prediction, target):
    df = pd.concat([prediction, target],axis=1)
    df.columns = ['prediction','target']
    df = df.sort_values(by = 'prediction', ascending = False)[:best_points]
    return (df['target'].sum() * income - budget)

### Прибыль по 1-му месторождению

In [16]:
revenue_1 = profit(pred_1, target_valid_1)

print('Прибыль для полученного объёма сырья региона №1 = {:.2f} руб'.format(revenue_1))

Прибыль для полученного объёма сырья региона №1 = 3320826043.14 руб


### Прибыль по 2-му месторождению

In [17]:
revenue_2 = profit(pred_2, target_valid_2)

print('Прибыль для полученного объёма сырья региона №2 = {:.2f} руб'.format(revenue_2))

Прибыль для полученного объёма сырья региона №2 = 2415086696.68 руб


### Прибыль по 3-му месторождению

In [18]:
revenue_3 = profit(pred_3, target_valid_3)

print('Прибыль для полученного объёма сырья региона №3 = {:.2f} руб'.format(revenue_3))

Прибыль для полученного объёма сырья региона №3 = 2710349963.60 руб


**Промежуточный вывод:**

Наибольшая прибыль в регионе №1 =  3320826043.14 руб (3 320 млн руб.)

## Расчёт прибыли и рисков 

* Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
* Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
* Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

In [19]:
def estimate(prediction, target):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        pred_subsample = prediction[target_subsample.index]
        values.append(profit(pred_subsample, target_subsample))
    values = pd.Series(values)
    mean = np.mean(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    confidence_interval = (lower, upper)
    risk_of_loss = (values < 0).sum() / values.count()
    
    print('Средняя прибыль = {:.2f} руб.'.format(mean))
    print('95% доверительный интервал от {:.2f} до {:.2f} руб.'.format(lower, upper))
    print('Процент риска {:.1%}'.format(risk_of_loss))

### Прибыль и риски по 1-му месторождению

In [20]:
region_1 = estimate(pred_1, target_valid_1)

Средняя прибыль = 396164984.80 руб.
95% доверительный интервал от -111215545.89 до 909766941.55 руб.
Процент риска 6.9%


### Прибыль и риски по 2-му месторождению

In [21]:
region_2 = estimate(pred_2, target_valid_2)

Средняя прибыль = 456045105.79 руб.
95% доверительный интервал от 33820509.40 до 852289453.87 руб.
Процент риска 1.5%


### Прибыль и риски по 3-му месторождению

In [22]:
region_3 = estimate(pred_3, target_valid_3)

Средняя прибыль = 404403866.57 руб.
95% доверительный интервал от -163350413.40 до 950359574.92 руб.
Процент риска 7.6%


**Промежуточный вывод:**


Регион с наименьшим риском во 2-м месторождении = 1.5%

А так же наибольшей средней прибылью 456045105.79 руб (456 млн руб.)

## Вывод

В результате исследования выбираем 2-ое местрождение с наименьшим процентом риска и наибольшой средней приболью.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: написана функция расчёта прибыли
- [x]  Выполнен шаг 5: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован